In [1]:
import Python as py
import os
import pandas as pd

In [3]:

import os
import pandas as pd
root = os.getcwd()
df_scheduled = pd.read_csv(root + '/data/scheduled_loan_repayments.csv')
df_actual = pd.read_csv(root + '/data/actual_loan_repayments.csv')
df_balances = py.calculate_df_balances(df_scheduled,df_actual)



In [4]:
def calculate_df_balances(df_scheduled,df_actual):
    """ 
        This is a utility function that creates a merged dataframe that will be used in the following questions. 
        This function will not be graded directly.

        Args:
            df_scheduled (DataFrame): Dataframe created from the 'scheduled_loan_repayments.csv' dataset
            df_actual (DataFrame): Dataframe created from the 'actual_loan_repayments.csv' dataset
        
        Returns:
            DataFrame: A merged Dataframe 

            Columns after the merge should be: 
            ['RepaymentID', 'LoanID', 'Month', 'ActualRepayment', 'LoanAmount', 'ScheduledRepayment']

            Additional columns to be used in later questions should include: 
            ['UnscheduledPrincipal', 'LoanBalanceStart, 'LoanBalanceEnd'] 
            Note: 'LoanBalanceStart' for the first month of each loan should equal the 'LoanAmount'

            You may create other columns to assist you in your calculations. e.g:
            ['InterestPayment']

    """

    #deleteing starts here
    df_merged = pd.merge(df_actual, df_scheduled)

    def calculate_balance(group):
        r_monthly = 0.1 / 12  
        group = group.sort_values('Month') 
        balances = []  
        interest_payments = []
        loan_start_balances = []
        for index, row in group.iterrows():
            if balances:
                interest_payment = balances[-1] * r_monthly
                balance_with_interest = balances[-1] + interest_payment
            else:
                interest_payment = row['LoanAmount'] * r_monthly
                balance_with_interest = row['LoanAmount'] + interest_payment
                loan_start_balances.append(row['LoanAmount'])

            new_balance = balance_with_interest - row['ActualRepayment']
            interest_payments.append(interest_payment)

            new_balance = max(0, new_balance)
            balances.append(new_balance)
            
        loan_start_balances.extend(balances)
        loan_start_balances.pop()
        group['LoanBalanceStart'] = loan_start_balances
        group['LoanBalanceEnd'] = balances
        group['InterestPayment'] = interest_payments
        return group

    df_balances = df_merged.groupby('LoanID').apply(calculate_balance).reset_index(drop=True)

    df_balances['LoanBalanceEnd'] = df_balances['LoanBalanceEnd'].round(2)
    df_balances['InterestPayment'] = df_balances['InterestPayment'].round(2)
    df_balances['LoanBalanceStart'] = df_balances['LoanBalanceStart'].round(2)
    df_balances['ScheduledPrincipal'] = df_balances['ScheduledRepayment'] - df_balances['InterestPayment']
    df_balances['UnscheduledPrincipal'] = np.where(df_balances['ActualRepayment'] > df_balances['ScheduledRepayment'], df_balances['ActualRepayment'] - df_balances['ScheduledRepayment'], 0)
    #deleteing ends here

    return df_balances

In [5]:
import numpy as np

calculate_df_balances(df_scheduled,df_actual)

,RepaymentID,LoanID,Month,ActualRepayment,LoanAmount,ScheduledRepayment,LoanBalanceStart,LoanBalanceEnd,InterestPayment,ScheduledPrincipal,UnscheduledPrincipal
0,544.0,1,1,2399.54,52000.0,2399.54,52000.00,50033.79,433.33,1966.21,0.0
1,1313.0,1,2,2399.54,52000.0,2399.54,50033.79,48051.20,416.95,1982.59,0.0
2,2551.0,1,3,2399.54,52000.0,2399.54,48051.20,46052.09,400.43,1999.11,0.0
3,3636.0,1,4,2399.54,52000.0,2399.54,46052.09,44036.32,383.77,2015.77,0.0
4,4300.0,1,5,2399.54,52000.0,2399.54,44036.32,42003.74,366.97,2032.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11995,7973.0,1000,8,2768.70,60000.0,2768.70,43716.52,41312.12,364.30,2404.40,0.0
11996,8115.0,1000,9,2768.70,60000.0,2768.70,41312.12,38887.69,344.27,2424.43,0.0
11997,9867.0,1000,10,2768.70,60000.0,2768.70,38887.69,36443.05,324.06,2444.64,0.0
11998,10128.0,1000,11,2768.70,60000.0,2768.70,36443.05,33978.05,303.69,2465.01,0.0
